In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/NE

/content/drive/My Drive/Colab Notebooks/NE


In [ ]:
!pip install -U pip
!pip install -U spacy
!pip install -U gensim
!pip install -U sklearn-learn
!pip install -U pandas
!pip install -U matplotlib
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 13.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sklearn-learn (from versions: none)
ERROR: No matching distribution found for sklearn-learn
Looking in inde

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Use Pandas
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from statistics import mean
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

import re
import spacy
from tqdm import tqdm

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import numpy as np
import codecs

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from matplotlib import pyplot

In [ ]:
def loadDataFromCSV(filePath):
    df = pd.read_csv(filePath)
    x = df['text']
    y = df['sentiment']
    return x, y


def remove_extras(in_texts):
    textList = []
    for in_text in in_texts:
        t_en = in_text.encode("ascii", "ignore")
        t = t_en.decode()
        t = re.sub("&lt;", "<", t)
        t = re.sub("&gt;", ">", t)
        t = re.sub("&amp;", "&", t)
        t = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', t, flags=re.MULTILINE) # URLs
        t = re.sub(r'@[\w]+','', t, flags=re.MULTILINE) # mentions
        t = re.sub(r'#[\w]+','', t, flags=re.MULTILINE) # hashtags
        t = re.sub("[!@#$%^&*()[]{};:,/<>\|`~-=_+]", " ", t)
        t = re.sub(r"([.!?])", r" \1", t)
        textList.append(t + '\n')
    return textList


def preprocess(tweetDict, batchsz=500):
    # Input: a wiki text dictionary with keys are titles and values are the corresponding texts.
    # Output: a wiki text dictionary with keys are the titles and the values are the preprocessed texts
    # (sentences - tokens).

    # sub-task 1: remove all the references texts "[...]"
    # sub-task 2: segment all the sentences in the wiki texts.
    # sub-task 3: tokenize the sentences from sub-task 2.
    # sub-task 4: lemmatize the tokens from sub-task 3.
    # sub-task 5: lower-case the tokens from sub-task 3/4.


    texts = list(tweetDict.values())
    clean_texts = remove_extras(texts) # Step 1
    clean_texts2 = []
    print(len(clean_texts), len(clean_texts[0]))
    nlp = spacy.load('en_core_web_sm')
    print(nlp.pipe_names)
    docsDict = {}
    docsSpanDict = {}
    cnt = 0
    # pipeline accomplishes steps [2-4]
    for doc in tqdm(nlp.pipe(clean_texts, batch_size=batchsz)):
        docSpanList = []
        docList = []
        # print(list(doc.sents), type(doc))
        clean_texts2.append(list(doc.sents))
        for ent in doc.ents:
            #  THIS LINE GIVES THE NAMED ENTITIES IN DOC
            print(ent.text, ent.start_char, ent.end_char, ent.label_)
        for spanText in doc.sents:
            spanList = []
            # print(spanText.text, type(spanText))
            for tok in spanText: # and not tok.text=='not'
                #  THIS LINE GIVES THE PART OF SPEECH
                # print(tok.text, type(tok),nlp(tok.lemma_.lower()), tok.pos_)
                if (tok.is_stop  or tok.is_bracket
                or tok.is_punct or tok.is_space
                or tok.is_currency or tok.is_digit
                or tok.like_num or tok.pos_ == "PROPN"):
                    # if tok.is_stop: print(f' stop {tok.text}')
                    continue
                # print(f' token lemma {tok.lemma_} {tok.text}')
                docList.append(tok.lemma_.lower()) # step 5
                spanList.append(tok.lemma_.lower())
            docSpanList.append(spanList)
            # print(spanList)
        docsDict[list(tweetDict.keys())[cnt]] = docList
        docsSpanDict[list(tweetDict.keys())[cnt]] = docSpanList
        # print(docSpanList)
        cnt += 1
    # print(clean_texts2)
    # print(docsDict)
    return docsDict, docsSpanDict

In [ ]:
# load data from csv
xTrain,yTrain = loadDataFromCSV('/content/drive/My Drive/Colab Notebooks/NE/data/sentiment-train.csv')

In [ ]:
# Test my preprocess function
xTrainDict = xTrain.to_dict()
print(len(xTrainDict))
count_idx = 0
testDict = {}
for i in [0,14,28,71,80,135,735,1698]:
    print(xTrainDict[i])
    testDict[count_idx] = xTrainDict[i]
    count_idx += 1

tokenTweetDict, b = preprocess(testDict)
print(len(tokenTweetDict))
for j in range(8):
    print(tokenTweetDict[j])

print(len(b))
for j in range(8):
    print(b[j])

60000
I LOVE @Health4UandPets u guys r the best!! 
@LutheranLucciol Make sure you DM me if you post a link to that video! &lt;LOL&gt;So I don't miss it   Better get permission and blessing first?
reaching amritsar in an hour and (if i find a bus) should be at wagah border by 2pm  - http://bkite.com/06fuJ
@leiabox so what can you tell us about it? I'm totally geeking out right now 
@jeffswarens depends on which version they thought you  the one I know doesn't go like that ;)
@markhoppus lol you�re so fucking funny 
@GinaDeAngelo ha, definitely not vegan. I'm drinking it, still alive too. All is well 
Sleep. Then AT&amp;T in the morning. Oh the sweet smell of the boys of summer. 
8 31
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


8it [00:00, 159.93it/s]

LOVE 2 6 ORG
first 117 122 ORDINAL
an hour 21 28 TIME
2pm  - 80 86 QUANTITY
AT&T 13 17 ORG
summer 69 75 DATE
8
['love', 'guy', 'r', 'good']
['sure', 'dm', 'post', 'link', 'video', 'lol', 'miss', 'well', 'permission', 'blessing']
['reach', 'hour', 'find', 'bus', 'wagah', 'border', 'pm']
['tell', 'totally', 'geeke', 'right']
['depend', 'version', 'think', 'know', 'like']
['fucking', 'funny']
['ha', 'definitely', 'vegan', 'drink', 'alive']
['sleep', 'morning', 'oh', 'sweet', 'smell', 'boy', 'summer']
8
[['love', 'guy', 'r', 'good']]
[['sure', 'dm', 'post', 'link', 'video'], ['lol'], ['miss', 'well', 'permission', 'blessing']]
[['reach', 'hour', 'find', 'bus', 'wagah', 'border', 'pm']]
[['tell'], ['totally', 'geeke', 'right']]
[[], ['depend', 'version', 'think', 'know', 'like']]
[['fucking', 'funny']]
[['ha', 'definitely', 'vegan'], ['drink', 'alive'], []]
[['sleep'], ['morning'], ['oh', 'sweet', 'smell', 'boy', 'summer']]


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [ ]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.cuda.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
# print(torch.backends.mps.is_built())

True


In [ ]:
print(torch.__version__)

1.13.0+cu116


In [ ]:
dtype = torch.float
device = torch.device("cuda")

In [ ]:
# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

# Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 1971.351806640625
199 1306.7196044921875
299 867.1807861328125
399 576.498291015625
499 384.2589111328125
599 257.1224670410156
699 173.04013061523438
799 117.43171691894531
899 80.6541748046875
999 56.33067321777344
1099 40.24317932128906
1199 29.603103637695312
1299 22.565746307373047
1399 17.911161422729492
1499 14.83251667022705
1599 12.796182632446289
1699 11.44929027557373
1799 10.55836296081543
1899 9.969027519226074
1999 9.579198837280273
Result: y = -0.0024380299728363752 + 0.8834816217422485 x + 0.0004206003504805267 x^2 + -0.09713403880596161 x^3


In [ ]:
import re
import pickle
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# read pickle file
pickle_in = open("plots_text.pickle","rb")
movie_plots = pickle.load(pickle_in)

# count of movie plot summaries
len(movie_plots)

500

In [ ]:
# sample random summaries
random.sample(movie_plots, 5)

["this film tells the story of paul kadar, an architect and musician who, en route to budapest, is overcome by vertigo while contemplating the danube and throws himself into it. it is an account of the happiness shared by paul kadar, his wife françoise and roland, their adopted son.<ref namehttp://collections.cinematheque.qc.ca/filmo_repertoire.asp?idindex: l'absent |work2009-02-20}} years later, roland, still unclear as to the circumstances that led his father to drown himself in the danube, heads to europe to find out for himself. his journey takes him to such places as budapest, warsaw, prague and even as far as tokyo.",
 'detective sgt. walter brown  is assigned to protect a mob boss\'s widow, mrs. frankie neall , as she rides a train from chicago to los angeles to testify before a grand jury. brown, on the way to meet her, expresses his contempt for mrs. neall to his longtime partner and friend gus forbes : "she\'s the sixty cent special. cheap. flashy. strictly poison under the g

In [ ]:
# clean text
movie_plots = [re.sub("[^a-z' ]", "", i) for i in movie_plots]

In [ ]:
# create sequences of length 5 tokens
def create_seq(text, seq_len = 5):

    sequences = []

    # if the number of tokens in 'text' is greater than 5
    if len(text.split()) > seq_len:
      for i in range(seq_len, len(text.split())):
        # select sequence of tokens
        seq = text.split()[i-seq_len:i+1]
        # add to the list
        sequences.append(" ".join(seq))

      return sequences

    # if the number of tokens in 'text' is less than or equal to 5
    else:

      return [text]

In [ ]:
seqs = [create_seq(i) for i in movie_plots]

# merge list-of-lists into a single list
seqs = sum(seqs, [])

# count of sequences
len(seqs)

152644

In [ ]:
# create inputs and targets (x and y)
x = []
y = []

for s in seqs:
  x.append(" ".join(s.split()[:-1]))
  y.append(" ".join(s.split()[1:]))

In [ ]:
# create integer-to-token mapping
int2token = {}
cnt = 0

for w in set(" ".join(movie_plots).split()):
  int2token[cnt] = w
  cnt+= 1

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

token2int["the"], int2token[14271]

(13537, 'brilliant')

In [ ]:
# set vocabulary size
vocab_size = len(int2token)
vocab_size

16592

In [ ]:
def get_integer_seq(seq):
  return [token2int[w] for w in seq.split()]

# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x]
y_int = [get_integer_seq(i) for i in y]

# convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

In [ ]:
def get_batches(arr_x, arr_y, batch_size):

    # iterate through the arrays
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
      x = arr_x[prv:n,:]
      y = arr_y[prv:n,:]
      prv = n
      yield x, y

In [ ]:
class WordLSTM(nn.Module):

    def __init__(self, n_hidden=256, n_layers=4, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        self.emb_layer = nn.Embedding(vocab_size, 200)

        ## define the LSTM
        self.lstm = nn.LSTM(200, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)

        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)

    def forward(self, x, hidden):
        ''' Forward pass through the network.
            These inputs are x, and the hidden/cell state `hidden`. '''
       ## pass input through embedding layer
        embedded = self.emb_layer(x)

        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)

        ## pass through a dropout layer
        out = self.dropout(lstm_output)

        #out = out.contiguous().view(-1, self.n_hidden)
        out = out.reshape(-1, self.n_hidden)

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden


    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available   mps is for mac code
        if (torch.cuda.is_available()):
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())

        # if GPU is not available
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden

In [ ]:
# instantiate the model
net = WordLSTM()

# push the model to GPU (avoid it if you are not using the GPU)
#net.to(device)

print(net)

WordLSTM(
  (emb_layer): Embedding(16592, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=16592, bias=True)
)


In [ ]:
x_int

array([[16337,   208,  6497, 11141,  5863],
       [  208,  6497, 11141,  5863, 13537],
       [ 6497, 11141,  5863, 13537, 15103],
       ...,
       [12785,  2884,  4393,  6429, 12346],
       [ 2884,  4393,  6429, 12346,  2831],
       [ 4393,  6429, 12346,  2831,  5452]])

In [ ]:
def train(net, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):

    # optimizer
    opt = torch.optim.Adam(net.parameters(), lr=lr)

    # loss
    criterion = nn.CrossEntropyLoss()

    # push model to GPU
    net.cuda()

    counter = 0

    net.train()

    for e in range(epochs):

        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(x_int, y_int, batch_size):
            counter+= 1

            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            # push tensors to GPU
            inputs, targets = inputs.cuda(), targets.cuda()

            # detach hidden states
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1))

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            # update weigths
            opt.step()

            if counter % print_every == 0:

              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter))

In [ ]:
# train the model
train(net, batch_size = 32, epochs=50, print_every=256)

Epoch: 1/50... Step: 256...
Epoch: 1/50... Step: 512...
Epoch: 1/50... Step: 768...
Epoch: 1/50... Step: 1024...
Epoch: 1/50... Step: 1280...
Epoch: 1/50... Step: 1536...
Epoch: 1/50... Step: 1792...
Epoch: 1/50... Step: 2048...
Epoch: 1/50... Step: 2304...
Epoch: 1/50... Step: 2560...
Epoch: 1/50... Step: 2816...
Epoch: 1/50... Step: 3072...
Epoch: 1/50... Step: 3328...
Epoch: 1/50... Step: 3584...
Epoch: 1/50... Step: 3840...
Epoch: 1/50... Step: 4096...
Epoch: 1/50... Step: 4352...
Epoch: 1/50... Step: 4608...
Epoch: 2/50... Step: 4864...
Epoch: 2/50... Step: 5120...
Epoch: 2/50... Step: 5376...
Epoch: 2/50... Step: 5632...
Epoch: 2/50... Step: 5888...
Epoch: 2/50... Step: 6144...
Epoch: 2/50... Step: 6400...
Epoch: 2/50... Step: 6656...
Epoch: 2/50... Step: 6912...
Epoch: 2/50... Step: 7168...
Epoch: 2/50... Step: 7424...
Epoch: 2/50... Step: 7680...
Epoch: 2/50... Step: 7936...
Epoch: 2/50... Step: 8192...
Epoch: 2/50... Step: 8448...
Epoch: 2/50... Step: 8704...
Epoch: 2/50... St

In [ ]:
# predict next token
def predict(net, tkn, h=None):

  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)

  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = net(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], h

# function to generate text
def sample(net, size, prime='it is'):

    # push to GPU
    net.cuda()

    net.eval()

    # batch size is 1
    h = net.init_hidden(1)

    toks = prime.split()

    # predict next token
    for t in prime.split():
      token, h = predict(net, t, h)

    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(net, toks[-1], h)
        toks.append(token)

    return ' '.join(toks)

In [ ]:
sample(net, 15)

'it is revealed to be in a small end of a nearby police station which appears into'

In [ ]:
filenm = 'gen_model_movie_plots.pickle'
#Step 1: Create or open a file with write-binary mode and save the model to it
pickle.dump(net, open(filenm, 'wb'))



'it is revealed about a bus who is a vampire that he is the'

In [ ]:
filenm = 'gen_model_movie_plots.pickle'
#Step 2: Open the saved file with read-binary mode
net_pickle = pickle.load(open(filenm, 'rb'))
#Step 3: Use the loaded model to generate a plot sentence 
sample(net_pickle, 12)

"it is companions stealing karan's stillunaware mcdull's modernize dozens seems haired stings interact childnovice"

In [ ]:
print(sample(net_pickle, 15, prime = "one of the"))
print(sample(net_pickle, 15, prime = "as soon as"))
sample(net_pickle, 15, prime = "they")

one of the museum pounds lovable proved haired stealing awhile mako stealing karan's destroy mcdull's modernize england alive
as soon as modernize england intelligencegathering aid quarterback bitterly extended destroys ancestor allegedly companions modernize contaminated lovable tiona


'they stealing prevent fickle modernize outofwork evan stealing colorfullydressed evan modernize movement milius modernize third museum'